# Итоговый проект. Улицы. Объекты культурного наследия.

In [ ]:
!pip install numpy pandas geopandas mapclassify osmnx keplergl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922316 sha256=77605cfb226e196c9630837f445f5bb45a51dce944ecd703f3f4b6dfe7db8bbd
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl


In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox

from keplergl import KeplerGl

In [ ]:
# code snippets - enable_custom_widget_manager
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = "Невский район, Санкт-Петербург"  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/amishenev/itmo_urban/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/amishenev/itmo_urban/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [ ]:
district = ox.geocode_to_gdf(TERRITORY_NAME)
district.explore(tiles=TILES)

## Улицы

### Загрузка файла с улицами (Extract)

In [ ]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf_streets = gpd.read_file(STREETS_FILE_URL, mask=district)

### Обработка данных с улицами (Transform)

In [ ]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
gdf_streets = gdf_streets.dissolve(by='name')

### Сохранение слоя с улицами (Load)

In [ ]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry

# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)

gdf_streets.rename(columns={"name": "Название"}, inplace=True)
gdf_streets.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf_streets = gdf_streets.to_crs(4326)
gdf_streets.to_file('streets.geojson', driver='GeoJSON')

<ipython-input-18-05a2e9d7c4d7>:4: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_streets.explode(tiles=TILES)


geometry
name                                                                           
1-й Рабфаковский переулок 0   LINESTRING (30.46855 59.85902, 30.46875 59.858...
1-я дорога                0   LINESTRING (30.48205 59.87700, 30.48225 59.877...
11-я линия                0   LINESTRING (30.46272 59.85269, 30.46324 59.85256)
13-я линия                0   LINESTRING (30.46202 59.85198, 30.46300 59.85171)
2-й Рабфаковский переулок 0   LINESTRING (30.47597 59.85304, 30.47590 59.853...
...                                                                         ...
улица Шелгунова           10  LINESTRING (30.45274 59.86604, 30.45289 59.86604)
                          11  LINESTRING (30.45260 59.86641, 30.45248 59.86637)
улица Шотмана             0   LINESTRING (30.48609 59.90234, 30.48597 59.902...
улица Юннатов             0   LINESTRING (30.49369 59.83316, 30.49382 59.833...
                          1   LINESTRING (30.49403 59.83343, 30.49435 59.833...

[1293 rows x 1 columns]

In [ ]:
map = KeplerGl(height=500)
map.add_data(data=gdf_streets.copy(), name='streets')
map

KeplerGl(data={'streets': {'index': ['1-й Рабфаковский переулок', '1-я дорога', '11-я линия', '13-я линия', '2…

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [ ]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf_giop = gpd.read_file(KGIOP_FILE_URL, mask=district)

### Обработка объектов культурного наследия (Transform)

In [ ]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
gdf_giop['lon'] = gdf_giop.to_crs(3857).geometry.centroid.to_crs(4326).x
gdf_giop['lat'] = gdf_giop.to_crs(3857).geometry.centroid.to_crs(4326).y
gdf_giop

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,4072,Палевский жилмассив (19 домов),Жилой дом,1925-1928,"Обуховской Обороны пр., 95, корп. 6",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3387453.441 8376910.050),30.430012,59.897308
1,4051,—,Здание школы (с садом и оградой),1932-1933,"Большой Смоленский пр., 36, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3385184.415 8376455.066),30.409629,59.895258
2,3974,—,Октябрьская набережная,1932-1936,"На правом берегу р. Невы, от устья р. Утки до ...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3388501.848 8377901.347),30.439430,59.901774
3,3975,—,Терраса у завода им. Ломоносова,1926,На р. Неве по пр. Обуховской Обороны,объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Памятник,POINT (3389224.756 8373469.957),30.445924,59.881805
4,4008,—,Шлиссельбургский мост,1928-1929,Через Обводный канал по Обуховской обороны пр.,объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Памятник,POINT (3383805.278 8381795.019),30.397240,59.919310
...,...,...,...,...,...,...,...,...,...,...,...
244,9618,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 94, корпус 3, лит. В",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392198.323 8371250.714),30.472636,59.871800
245,9619,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 96, корпус 1, лит. М",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392241.960 8371116.095),30.473028,59.871193
246,9620,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 96, корпус 2, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392244.854 8371170.652),30.473054,59.871439
247,9621,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 96, корпус 3, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392259.882 8371238.738),30.473189,59.871746


### Сохранение слоя с объектами культурного наследия (Load)

In [ ]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
col_names = {
    'id': '',
    'ensemble_name': 'Названия ансамбля',
    'object_name': 'Название объекта',
    'occurrence_time': 'Год(ы) появления',
    'object_location': 'Адрес',
    'historical_category': 'Историческая категория',
    'normative_act': 'Нормативный акт',
    'object_type': 'Тип объекта',
    'lon': 'Долгота',
    'lat': 'Широта'
}
gdf_giop.rename(columns=col_names, inplace=True)
gdf_giop.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf_giop = gdf_giop.to_crs(4326)
gdf_giop.to_file('kgiop.geojson', driver='GeoJSON')

/usr/local/lib/python3.10/dist-packages/fiona/collection.py:246: FeatureWarning: Empty field name at index 0
  self.session.start(self, **kwargs)


In [ ]:
map = KeplerGl(height=500)
map.add_data(data=gdf_giop.copy(), name="GIOP")

map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'GIOP': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21…

In [ ]:
map = KeplerGl(height=500)
map.add_data(data=gdf_streets.copy(), name="streets")
map.add_data(data=gdf_giop.copy(), name="GIOP")

map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'streets': {'index': ['1-й Рабфаковский переулок', '1-я дорога', '11-я линия', '13-я линия', '2…

In [ ]:
map.save_to_html()

Map saved to keplergl_map.html!
